In [ ]:
!pip install roboflow

In [3]:
from roboflow import Roboflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="hiXB6MTQOH0hlxROddmK")
project = rf.workspace("jeevan-sendur-g-workspace").project("board-kzxer")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to board-2 in yolov8:: 100%|██████████| 980/980 [00:00<00:00, 3425.67it/s]


In [ ]:
! pip install ultralytics

In [6]:
from ultralytics import YOLO

model = YOLO("yolo11n-pose.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.97M/5.97M [00:00<00:00, 103MB/s]


In [7]:
results = model.train(data="/content/board-2/data.yaml", epochs=100, imgsz=640)

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolo11n-pose.pt, data=/content/board-2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

100%|██████████| 755k/755k [00:00<00:00, 18.7MB/s]


Overriding model.yaml kpt_shape=[17, 3] with kpt_shape=[4, 3]
Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True

100%|██████████| 5.35M/5.35M [00:00<00:00, 103MB/s]


AMP: checks passed ✅


...

    100/100      2.42G     0.2743     0.2889    0.04595     0.1581     0.8362         10        640: 100%|██████████| 27/27 [00:10<00:00,  2.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.73it/s]

                   all         28         28      0.998          1      0.995      0.974      0.998          1      0.995      0.992



100 epochs completed in 0.322 hours.
Optimizer stripped from runs/pose/train/weights/last.pt, 5.6MB
Optimizer stripped from runs/pose/train/weights/best.pt, 5.6MB

Validating runs/pose/train/weights/best.pt...
Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n-pose summary (fused): 257 layers, 2,654,479 parameters, 0 gradients, 6.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.73it/s]


                   all         28         28      0.998          1      0.995      0.978      0.998          1      0.995      0.991
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/pose/train


In [10]:
model = YOLO("/content/runs/pose/train/weights/best.pt")

results = model("/content/testingpic.jpg")


image 1/1 /content/testingpic.jpg: 640x480 1 board, 54.3ms
Speed: 3.2ms preprocess, 54.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)


In [13]:
for result in results:
    keypoints = result.keypoints
    result.save(filename="result.jpg")

In [16]:
import cv2
import numpy as np

def visualize_keypoints(results, thickness=4, radius=5, line_thickness=2, point_color=(0, 255, 0), line_color=(255, 255, 0)):
    for result in results:
        # Get the original image
        img = result.orig_img.copy()

        # Get keypoints
        keypoints = result.keypoints.data

        if len(keypoints):
            for kpts in keypoints:
                # Store valid keypoints
                valid_points = []

                # Draw keypoints and collect valid ones
                for x, y, conf in kpts:
                    if conf > 0:
                        x_coord, y_coord = int(x), int(y)
                        valid_points.append((x_coord, y_coord))
                        cv2.circle(img, (x_coord, y_coord), radius, point_color, thickness)

                # Connect points in a square if we have 4 points
                if len(valid_points) == 4:
                    # Connect points in square formation
                    for i in range(4):
                        # Connect to next point (last point connects to first)
                        next_idx = (i + 1) % 4
                        cv2.line(img, valid_points[i], valid_points[next_idx],
                                line_color, line_thickness)

        # Save the result
        cv2.imwrite('keypoints_square.jpg', img)

# Usage:
for result in results:
    visualize_keypoints(
        results=[result],
        thickness=7,      # Keypoint circle thickness
        radius=5,         # Keypoint circle radius
        line_thickness=6, # Thickness of connecting lines
        point_color=(0, 255, 0),   # Green dots
        line_color=(255, 255, 0)   # Yellow lines
    )